In [ ]:
import torchvision.models as models
import mil_dsmil_softmax as mil

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable

import sys
import argparse
import os
import pandas as pd
import csv
import numpy as np
import random
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_fscore_support
from sklearn.datasets import load_svmlight_file
from PIL import Image
from collections import OrderedDict
import copy
import itertools

import torchvision.transforms.functional as VF
from torchvision import transforms

os.environ['CUDA_VISIBLE_DEVICES']='0'
use_cuda = torch.cuda.is_available()
if use_cuda:
    torch.cuda.set_device(0)

In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self, in_channel, output_class=1):
        super(FeatureExtractor, self).__init__()
        self.fc = nn.Linear(in_channel, output_class)
        
    def forward(self, x):
        c = self.fc(x)
        return x, c

In [ ]:
def get_data(file_path):
    df = pd.read_csv(file_path)
    df = pd.DataFrame(df)
    df = df[df.columns[0]]
    data_list = []    
    for i in range(0, df.shape[0]):  
        data = str(df.iloc[i]).split(' ')
        ids = data[0].split(':')
        idi = int(ids[0])
        idb = int(ids[1])
        idc = int(ids[2])
        data = data[1:]
        feature_vector = np.zeros(len(data))  
        for i, feature in enumerate(data):
            feature_data = feature.split(':')
            if len(feature_data) == 2:
                feature_vector[i] = feature_data[1]
        data_list.append([idi, idb, idc, feature_vector])
    return data_list

In [ ]:
def get_bag(data, idb):
    data_array = np.array(data)
    bag_id = data_array[:, 1]
    return data_array[np.where(bag_id == idb)]

In [ ]:
def epoch_train(bag_ins_list):
    epoch_loss = 0
    for i, data in enumerate(bag_ins_list):
        optimizer.zero_grad()
        data_bag_list = shuffle(data[1])
        data_tensor = torch.from_numpy(np.stack(data_bag_list)).float().cuda()
        data_tensor = data_tensor[:, 0:num_feats]
        label = torch.from_numpy(np.array(int(np.clip(data[0], 0, 1)))).float().cuda()
#         print(label)
        classes, bag_prediction, _ = milnet(data_tensor) # n X L
#         print(A, 1)
        max_prediction, index = torch.max(classes, 0)
        loss_bag = mse_loss(bag_prediction.view(1, -1), label.view(1, -1))
        loss_max = mse_loss(max_prediction.view(1, -1), label.view(1, -1))
        loss_total = 0.75*loss_bag + 0.25*loss_max
        loss_total = loss_total.mean()
        loss_total.backward()
        optimizer.step()  
        epoch_loss = epoch_loss + loss_total.item()
    return epoch_loss / len(bag_ins_list)

In [ ]:
def epoch_test(bag_ins_list):
    bag_labels = []
    bag_predictions = []
    epoch_loss = 0
    with torch.no_grad():
        for i, data in enumerate(bag_ins_list):
            bag_labels.append(np.clip(data[0], 0, 1))
            data_tensor = torch.from_numpy(np.stack(data[1])).float().cuda()
            data_tensor = data_tensor[:, 0:num_feats]
            label = torch.from_numpy(np.array(int(np.clip(data[0], 0, 1)))).float().cuda()
            classes, bag_prediction, _ = milnet(data_tensor) # n X L
            max_prediction, index = torch.max(classes, 0)
            loss_bag = mse_loss(bag_prediction.view(1, -1), label.view(1, -1))
            loss_max = mse_loss(max_prediction.view(1, -1), label.view(1, -1))
            loss_total = 0.75*loss_bag + 0.25*loss_max
            loss_total = loss_total.mean()
#             bag_predictions.append(0.75*bag_prediction.cpu().squeeze().numpy() + 0.25*max_prediction.cpu().squeeze().numpy())
            bag_predictions.append(torch.sigmoid(bag_prediction).cpu().squeeze().numpy())
            epoch_loss = epoch_loss + loss_total.item()
    epoch_loss = epoch_loss / len(bag_ins_list)
    return epoch_loss, bag_labels, bag_predictions

In [ ]:
def optimal_thresh(fpr, tpr, thresholds, p=0):
    loss = (fpr - tpr) - p * tpr / (fpr + tpr + 1)
    idx = np.argmin(loss, axis=0)
    return fpr[idx], tpr[idx], thresholds[idx]

In [ ]:
def five_scores(bag_labels, bag_predictions):
    fpr, tpr, threshold = roc_curve(bag_labels, bag_predictions, pos_label=1)
    fpr_optimal, tpr_optimal, threshold_optimal = optimal_thresh(fpr, tpr, threshold)
    auc_value = roc_auc_score(bag_labels, bag_predictions)
    this_class_label = np.array(bag_predictions)
    this_class_label[this_class_label>=threshold_optimal] = 1
    this_class_label[this_class_label<threshold_optimal] = 0
    bag_predictions = this_class_label
    precision, recall, fscore, _ = precision_recall_fscore_support(bag_labels, bag_predictions, average='binary')
    accuracy = 1- np.count_nonzero(np.array(bag_labels).astype(int)- bag_predictions.astype(int)) / len(bag_labels)
#     print(bag_labels)
#     print(bag_predictions)
    return accuracy, auc_value, precision, recall, fscore

In [ ]:
def cross_validation_set(in_list, fold, index):
    csv_list = copy.deepcopy(in_list)
    n = int(len(csv_list)/fold)
    chunked = [csv_list[i:i+n] for i in range(0, len(csv_list), n)]
    test_list = chunked.pop(index)
    return list(itertools.chain.from_iterable(chunked)), test_list

In [ ]:
def compute_pos_weight(bags_list):
    pos_count = 0
    for item in bags_list:
        pos_count = pos_count + np.clip(item[0], 0, 1)
    return (len(bags_list)-pos_count)/pos_count

In [ ]:
### Musk 1
data_all = get_data('mil_dataset/Musk/musk2norm.svm')

In [ ]:
data_all

In [ ]:
bag_ins_list = []
num_bag = data_all[-1][1]+1
# print(num_bag)
for i in range(num_bag):
    bag_data = get_bag(data_all, i)
    bag_label = bag_data[0, 2]
    bag_vector = bag_data[:, 3]
    print('bag')
    print(bag_data.shape)
    bag_ins_list.append([bag_label, bag_vector])
bag_ins_list = shuffle(bag_ins_list)
# print(bag_ins_list)
print(len(bag_ins_list))
bags_list, test_list = cross_validation_set(bag_ins_list, fold=10, index=1)
# print(len(test_list))
print(len(bags_list))

In [ ]:
test_list

In [ ]:
torch.tensor(compute_pos_weight(bags_list))

In [ ]:
acs = []
num_feats = 166
for k in range(0, 10):
    bags_list, test_list = cross_validation_set(bag_ins_list, fold=10, index=k)
    i_classifier = FeatureExtractor(num_feats, 1)
    b_classifier = mil.BClassifier(input_size=num_feats, output_class=1)
    milnet = mil.MILNet(i_classifier, b_classifier).cuda()
#     mse_loss = nn.MSELoss()
    pos_weight = torch.tensor(compute_pos_weight(bags_list))
    mse_loss = nn.BCEWithLogitsLoss(pos_weight)
    optimizer = torch.optim.Adam(b_classifier.parameters(), lr=0.0001, betas=(0.5, 0.9), weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 50, 0.1)
    optimal_ac = 0
    for epoch in range(0, 100):
        train_loss = epoch_train(bags_list) # iterate all bags
#         print('\r[%d/%d] train loss: %.4f' % (epoch, 100, train_loss))
        test_loss, bag_labels, bag_predictions = epoch_test(test_list)
        accuracy, auc_value, precision, recall, fscore = five_scores(bag_labels, bag_predictions)
#         print('\r        test loss: %.4f, average score: %.4f, aug score: %.4f, precision: %.4f, recall: %.4f, fscore: %.4f' % 
#               (test_loss, accuracy, auc_value, precision, recall, fscore))

        optimal_ac = max(accuracy, optimal_ac)

        scheduler.step()
    print('Optimal ac: %.4f' % 
              (optimal_ac))
    acs.append(optimal_ac)
print('mean: %.4f, std %.4f' % (np.mean(np.array(acs)), np.std(np.array(acs))))

In [ ]:
A = torch.from_numpy(np.array([[1, 5, 4, 6], [2, 2, 4, 4]]))
B = torch.from_numpy(np.array([1, 2, 3, 4]))
# _, indices = torch.sort(A, 0, True) 
# B = torch.index_select(B, 0, indices)
print(A * B)